# Pregunta 1

### Librerías necesarias

In [205]:
from Crypto.Cipher import AES

### Función de compresión Davies-Meyer

In [206]:
def davies_meyer(encrypt, l_key, l_message):
    def prev(message):
        key_bits = message[0:l_key]                #Separamos el mensaje según el largo de la llave
        msg_bits = message[l_key:]
        key = int.from_bytes(key_bits, byteorder='big')                #Pasamos la llave y el mensaje a enteros
        msg = int.from_bytes(msg_bits, byteorder='big')
        cript = int.from_bytes(encrypt(key_bits, msg_bits), byteorder='big')                #Encriptamos con la función entregada
        res = cript ^ msg                #Ejectuamos xor entre el encriptado y el mensaje
        res_bytes = res.to_bytes(l_message, byteorder='big')                #Pasamos resultado a bytes
        return bytearray(res_bytes)
    return prev

#### Ejemplo con AES:

In [207]:
def AES_128(key: bytearray, message: bytearray) -> bytearray:
    a = AES.new(key, AES.MODE_ECB)
    return bytearray(a.encrypt(message))

In [208]:
ejemplo = davies_meyer(AES_128, 16, 16)
print(ejemplo(bytearray(b'01234567890123440123456789012344')))

bytearray(b'\x17L\xb4\xe7\xf6\xcew\xcc\xe8\xba\x12F\xb7\xd4\xb6r')


### Padding

In [209]:
def pad(message, l_block):
    l_message = len(message)                #Largo del mensaje
    l_message_byte = bytearray(l_message.to_bytes(l_block, "big"))                #Mensaje a bytearray
    l_padding = 0
    if l_message % l_block != 0 and l_message > 0:                #Revisamos si el largo del mensaje es múltiplo del largo del bloque o es 0
        l_padding = l_block - (l_message % l_block)
    elif l_message % l_block != 0 and l_message == 0:
        l_padding = l_block
    zero = 0
    one = 1
    message_cpy = message.copy()
    for i in range(l_padding):                #Agregamos padding según lo necesario
        if i == 0:
            message_cpy += one.to_bytes(1, byteorder="big")                #1 inicial en el padding
        else:
            message_cpy += (zero.to_bytes(1, byteorder='big'))
    message_cpy += (l_message_byte)                #Largo del mensaje en el bloque final
    return message_cpy

#### Ejemplo de padding

In [210]:
print(pad(bytearray(b'012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789'), 16))

bytearray(b'012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01,')


### Merkle-Damgard

In [211]:
def merkle_damgard(IV, comp, l_block):
    def compression(message):
        H = IV                #Definimos H como H0 entregado
        msg = pad(message, l_block)                #aplicamos pad al mensaje a comprimir
        l_message = len(msg)
        rounds = int(len(msg)/l_block)                #Calculamos la cantidad de rondas según el largo del mensaje y los bloques
        for i in range (0, rounds):                #Por cada ronda:
            union = msg[(i)*l_block:(i+1)*l_block] + H                #Unimos el trozo correspondiente del mensaje con el H
            H = comp(union)                #Actualizamos el H utilizando la unión y la función de compresión.
        return H                #Finalmente, retornemos el mensaje
    return compression                #Retornamos la función

#### Ejemplo de prueba con AES y largo 16

In [212]:
comp = davies_meyer(AES_128, 16, 16)
hash = merkle_damgard(bytearray (b'1234567890123456'), comp, 16)
s1 = bytearray(b'Este es un mensaje de prueba para la tarea 2')
s2 = bytearray(b'Este es un mensaje de Prueba para la tarea 2')
s3 = bytearray(b'Un mensaje corto')
s4 = bytearray(b'')

h1 = hash(s1)
h2 = hash(s2)
h3 = hash(s3)
h4 = hash(s4)

print(h1)
print(h2)
print(h3)
print(h4)

bytearray(b'\xe9\xe8\xac\x12\\\xf2\xc8\x16\xceOV\xc5Y.T\xea')
bytearray(b'\xb6\xfb\xc6a\x12\xae\x95\x1f\xda\xc5\x13\xde\x06|Q\x96')
bytearray(b'\xc5\xec\xcdd\xa4(R*\xf0L*QtL\xda\x81')
bytearray(b'p\xca \xd8\x9c\xeb\xe6\xb1\xce\xcf\x03\xb2\x9e\x93\x19\xbc')
